In [1]:
import torch
import numpy as np
import os
import json
from torchvision.io.video import read_video
from transformers import AutoProcessor, VideoMAEModel
from qwen_vl_utils import process_vision_info
from vllm import LLM, SamplingParams

# --- [1. 설정 및 VideoMAE 로드] ---
device = "cuda" if torch.cuda.is_available() else "cpu"

# VideoMAE (Retriever)
retriever_model_id = "MCG-NJU/videomae-base" 
feature_extractor = AutoProcessor.from_pretrained(retriever_model_id)
retriever_model = VideoMAEModel.from_pretrained(retriever_model_id).to(device)


/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [2]:
def get_video_embedding(video_path):
    """VideoMAE를 사용해 영상 특징 추출 (전처리 포함)"""
    try:
        rgb, audio, info = read_video(video_path, pts_unit='sec')
    except Exception as e:
        print(f"Error reading {video_path}: {e}")
        return torch.zeros(1, 768).cpu()

    # 프레임 샘플링 (16개)
    if rgb.size(0) > 16:
        indices = torch.linspace(0, rgb.size(0) - 1, 16).long()
        video_frames = rgb[indices]
    else:
        indices = torch.linspace(0, rgb.size(0) - 1, 16).long()
        video_frames = rgb[indices]

    video_frames_numpy = list(video_frames.numpy())
    inputs = feature_extractor(video_frames_numpy, return_tensors="pt")
    
    with torch.no_grad():
        outputs = retriever_model(**inputs.to(device))
        video_emb = outputs.last_hidden_state.mean(dim=1) 
        
    video_emb = video_emb / video_emb.norm(p=2, dim=-1, keepdim=True)
    return video_emb.cpu()

def rag_inference(new_video_path, database_matrix):
    """가장 유사한 영상 1개의 인덱스 반환"""
    query_emb = get_video_embedding(new_video_path)
    similarity = torch.mm(query_emb, database_matrix.t())
    scores, indices = torch.topk(similarity, k=1)
    return indices[0].item() # .item()으로 스칼라 값 반환

In [3]:
# --- [2. Database Indexing (VideoMAE)] ---
print("Indexing Database...")

video_path = os.path.join(os.getcwd(), "datasets", "videos")
video_path_list = os.listdir(video_path)
# JSON 로드 (경로는 실제 환경에 맞게 수정 필요)
text_json = json.load(open("./datasets/train_video_reason_annotation.json"))

# 빠른 검색을 위한 매핑
video_to_conversations = {}
for item in text_json:
    json_video_name = os.path.basename(item["video"])
    video_to_conversations[json_video_name] = item["conversations"]

dataset_embeddings = []
dataset_embeddings_video_name = []
dataset_context_map = {} # 인덱스 -> 텍스트 매핑

for i, video_name in enumerate(video_path_list):
    full_path = os.path.join(video_path, video_name)
    emb = get_video_embedding(full_path)
    dataset_embeddings.append(emb)
    dataset_embeddings_video_name.append(video_name)
    
    # 해당 영상의 'GPT 답변'만 추출하여 Context로 저장
    if video_name in video_to_conversations:
        convs = video_to_conversations[video_name]
        # 보통 conversations 구조: [{'from': 'human', ...}, {'from': 'gpt', 'value': '정답 텍스트'}]
        # GPT의 답변을 합쳐서 문맥으로 사용
        context_text = " ".join([turn['value'] for turn in convs if turn['from'] == 'gpt'])
        dataset_context_map[i] = context_text
    else:
        dataset_context_map[i] = "No description available."

dataset_embeddings = torch.cat(dataset_embeddings)
# 전체 정규화
dataset_embeddings = dataset_embeddings / dataset_embeddings.norm(p=2, dim=-1, keepdim=True)

# --- [3. vLLM (Qwen-VL) 설정] ---
os.environ['VLLM_WORKER_MULTIPROC_METHOD'] = 'spawn'


Indexing Database...


/usr/local/lib/python3.12/dist-packages/torchvision/io/_video_deprecation_warning.py:9: UserWarning: The video decoding and encoding capabilities of torchvision are deprecated from version 0.22 and will be removed in version 0.24. We recommend that you migrate to TorchCodec, where we'll consolidate the future decoding/encoding capabilities of PyTorch: https://github.com/pytorch/torchcodec
  warnings.warn(


In [4]:
# vLLM 입력 준비 함수
def prepare_inputs_for_vllm(messages, processor):
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    
    # Qwen2-VL 특화 처리 (Dynamic Resolution 등)
    image_inputs, video_inputs, video_kwargs = process_vision_info(
        messages,
        image_patch_size=processor.image_processor.patch_size,
        return_video_kwargs=True,
        return_video_metadata=True
    )
    
    mm_data = {}
    if image_inputs is not None:
        mm_data['image'] = image_inputs
    if video_inputs is not None:
        mm_data['video'] = video_inputs

    return {
        'prompt': text,
        'multi_modal_data': mm_data,
        "mm_processor_kwargs": video_kwargs, # 중요: Qwen2-VL의 grid thw 정보 전달
    }

# 모델 로드 (경로 주의)
checkpoint_path = os.path.join(os.getcwd(), "weights", "Qwen3-VL-4B-Thinking-FP8")
# Qwen-VL용 Processor 로드
processor = AutoProcessor.from_pretrained(checkpoint_path, trust_remote_code=True)

llm = LLM(
    model=checkpoint_path,
    trust_remote_code=True,
    gpu_memory_utilization=0.80, # 필요시 조절
    max_model_len=8192,
    enforce_eager=True,
    tensor_parallel_size=torch.cuda.device_count(),
    seed=0,
    # Qwen2-VL 계열은 limit_mm_per_prompt 등을 설정해야 할 수 있음 (최신 버전은 자동)
)



INFO 12-15 10:11:47 [utils.py:253] non-default args: {'trust_remote_code': True, 'max_model_len': 8192, 'gpu_memory_utilization': 0.8, 'disable_log_stats': True, 'enforce_eager': True, 'model': '/root/backup/workspace/RAG-example/weights/Qwen3-VL-4B-Thinking-FP8'}


The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.


INFO 12-15 10:11:47 [model.py:637] Resolved architecture: Qwen3VLForConditionalGeneration
INFO 12-15 10:11:47 [model.py:1750] Using max model len 8192


2025-12-15 10:11:48,070	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 12-15 10:11:48 [scheduler.py:228] Chunked prefill is enabled with max_num_batched_tokens=8192.
WARNING 12-15 10:11:48 [vllm.py:601] Enforce eager set, overriding optimization level to -O0
INFO 12-15 10:11:48 [vllm.py:707] Cudagraph is disabled under eager mode
(EngineCore_DP0 pid=3992004) INFO 12-15 10:11:52 [core.py:93] Initializing a V1 LLM engine (v0.12.0) with config: model='/root/backup/workspace/RAG-example/weights/Qwen3-VL-4B-Thinking-FP8', speculative_config=None, tokenizer='/root/backup/workspace/RAG-example/weights/Qwen3-VL-4B-Thinking-FP8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, data_parallel_size=1, disable_custom_all_reduce=False, quantization=fp8, enforce_eager=True, kv_cache_dtype=auto, device_config=cuda, structured_outputs_config=StructuredOutputsConfig(backend='auto'

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  8.28it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  8.27it/s]
(EngineCore_DP0 pid=3992004) 


(EngineCore_DP0 pid=3992004) INFO 12-15 10:11:56 [gpu_model_runner.py:3549] Model loading took 5.2960 GiB memory and 0.296728 seconds
(EngineCore_DP0 pid=3992004) INFO 12-15 10:11:56 [gpu_model_runner.py:4306] Encoder cache will be initialized with a budget of 151250 tokens, and profiled with 1 video items of the maximum feature size.
(EngineCore_DP0 pid=3992004) WARNING 12-15 10:12:01 [fp8_utils.py:777] Using default W8A8 Block FP8 kernel config. Performance might be sub-optimal! Config file not found at /usr/local/lib/python3.12/dist-packages/vllm/model_executor/layers/quantization/utils/configs/N=6144,K=2560,device_name=NVIDIA_GeForce_RTX_4080,dtype=fp8_w8a8,block_shape=[128,128].json
(EngineCore_DP0 pid=3992004) WARNING 12-15 10:12:01 [fp8_utils.py:777] Using default W8A8 Block FP8 kernel config. Performance might be sub-optimal! Config file not found at /usr/local/lib/python3.12/dist-packages/vllm/model_executor/layers/quantization/utils/configs/N=2560,K=4096,device_name=NVIDIA_Ge

In [8]:
sampling_params = SamplingParams(
    temperature=0.1, # 사실적 분석을 위해 낮게 설정
    max_tokens=256,
    stop=["<|endoftext|>", "<|im_end|>"]
)

# --- [4. RAG + vLLM Inference Pipeline] ---
print("Start Inference...")
new_video_path = os.path.join(os.getcwd(), "datasets", "val", "mounting")
video_list = os.listdir(new_video_path)

FP = 0 # False Positive Count

# 배치 처리를 원하면 리스트에 모아서 llm.generate에 한 번에 넘길 수도 있음
# 여기서는 로직 확인을 위해 Loop 방식 유지
for video_name in video_list:
    target_full_path = os.path.join(new_video_path, video_name)
    
    # [Step 1] RAG: 유사한 과거 영상 찾기
    idx = rag_inference(target_full_path, dataset_embeddings)
    
    # [Step 2] Context 추출
    retrieved_video_name = dataset_embeddings_video_name[idx]
    retrieved_context = dataset_context_map[idx]
    
    # 디버깅: 검색된 영상 정보
    # print(f"Target: {video_name} -> Similar to: {retrieved_video_name}")
    
    # [Step 3] Prompt 구성 (RAG 적용)
    # 검색된 'retrieved_context'를 프롬프트에 주입하여 모델이 참고하게 함
    # system_msg = "You are an expert in analyzing livestock behavior from videos."
    user_msg = (
        f"Context: I found a similar historical video which was described as follows: \"{retrieved_context}\"\n\n"
        "Task: Based on the visual content of the provided video and the context above, "
        "analyze whether the cow's behavior is Normal or Abnormal. "
        "Provide a reasoning and conclude with 'Status: Normal' or 'Status: Abnormal'."
    )
    
    messages = [
        {
            "role": "user", 
            "content": [
                {
                    "type": "video", 
                    "video": target_full_path,
                    "max_pixels": 360 * 420, # 필요시 조절
                    "fps": 1.0, 
                },
                {"type": "text", "text": user_msg},
            ]
        }
    ]
    
    # [Step 4] vLLM Inference
    try:
        inputs = prepare_inputs_for_vllm(messages, processor)
        
        # llm.generate는 요청 리스트를 받습니다.
        outputs = llm.generate([inputs], sampling_params=sampling_params, use_tqdm=False)
        generated_text = outputs[0].outputs[0].text
        
        print(f"Video: {video_name}")
        print(f"Ref Video: {retrieved_video_name}")
        print(f"Output: {generated_text}\n" + "-"*30)
        
        # [Step 5] 결과 분석 (False Positive Check)
        # "Normal"이라고 판단했는지 확인 (대소문자 무시)
        if "Normal" in generated_text or "status: normal" in generated_text.lower():
            # 만약 실제 정답이 Abnormal이어야 하는데 Normal이라고 했다면 FP (혹은 정의에 따라 다름)
            # 여기서는 코드 작성자분의 의도(특정 단어 포함 여부 카운트)를 따름
            FP += 1
            
    except Exception as e:
        print(f"Inference Error on {video_name}: {e}")

    break

print(f"Total FP Count: {FP}")

Start Inference...


[2025-12-15 10:17:26] INFO vision_process.py:320: decord:  video_path='/root/backup/workspace/RAG-example/datasets/val/mounting/crop203.mp4', total_frames=128, video_fps=15.118110236220472, time=0.024s


Inference Error on crop203.mp4: EngineCore encountered an issue. See stack trace (above) for the root cause.
Total FP Count: 0
